#### ASCII Operators

Following Python functions are from the course notes:

In [39]:
def runwasm(wasmfile):
    from IPython.display import display, Javascript
    display(Javascript("""
    const params = {
        P0lib: {
            write: i => element.append(i + ' '),
            writeln: () => element.append(document.createElement('br')),
            read: () => window.prompt()
        }
    }
    
    var wasmByteString = \"""" + str(open(wasmfile, "rb").read()) + """\"; // pass the wasm file to Js by byte string
    wasmByteString = wasmByteString.substring(2, wasmByteString.length - 1); // Remove the byte literals b'...'
    const wasmArrayBuffer = new Uint8Array(wasmByteString.length); // convert the binery string to ArrayBuffer
    for (let i = 0; i < wasmByteString.length; i++)
      wasmArrayBuffer[i] = wasmByteString.charCodeAt(i);
    
    WebAssembly.compile(wasmArrayBuffer.buffer) // compile (sharable) code.wasm
        .then(module => WebAssembly.instantiate(module, params)) // create an instance with memory
        // .then(instance => instance.exports.program()); // run the main program; not needed if start function specified
     """))

In [40]:
def runpywasm(wasmfile):
    import pywasm
    def write(s, i): print(i, end=' ')
    def writeln(s): print()
    def read(s): return int(input())
    vm = pywasm.load(wasmfile, {'P0lib': {'write': write, 'writeln': writeln, 'read': read}})

In [41]:
from wasmer import engine, Store, Module, Instance, ImportObject, Function
from wasmer_compiler_cranelift import Compiler

def runwasmer(wasmfile):
    def write(i: int): print(i, end=' ')
    def writeln(): print()
    def read() -> int: return int(input()) 
    store = Store(engine.JIT(Compiler))
    module = Module(store, open(wasmfile, 'rb').read())
    import_object = ImportObject()
    import_object.register("P0lib", {"write": Function(store, write),
                                     "writeln": Function(store, writeln),"read": Function(store, read)})
    instance = Instance(module, import_object)

In [42]:
import nbimporter; nbimporter.options["only_defs"] = False
from P0 import compileString

Extend the P0 compiler with following ASCII alternatives to Unicode characters:

- `not` for `¬`
- `->`, `<-` for `←`, `→`

Modify the P0 compiler that comes with this lab such that both Unicode characters and ASCII equivalents are allowed. Give the modifications to the grammar in the next cell. A test case is below:

### Parts of the P0 compiler that are modified:
- In SC.ipynb, a new keyword 'not' mapping to value `NOT` is added the map of keywords
```python
KEYWORDS = \
{'div': DIV, 'mod': MOD, 'and': AND, 'or': OR, 'if': IF, 'then': THEN,
 'else': ELSE, 'while': WHILE, 'do': DO, 'const': CONST, 'type': TYPE,
 'var': VAR, 'set': SET, 'procedure': PROCEDURE, 'program': PROGRAM, 'not': NOT}
```
- In the `getSym()` function of SC.ipynb, an additional check for symbol `>` following the minus sign `-` is added to parse the right arrow `->`
```python
elif ch == '-': 
    getChar()
    if ch == '>': getChar(); sym = RARROW
    else: sym = MINUS
```
- In the `getSym()` function of SC.ipynb, an additional check for symbol `-` following the less than sign `<` is added to parse the left arrow `<-`
```python
elif ch == '<': 
    getChar()
    if ch == '-': getChar(); sym = LARROW
    else: sym = LT
```

### Modified P0 Grammar

    selector ::= { "[" expression "]" | "." ident}
    factor ::= ident selector | integer | "(" expression ")" | "{" [expression {"," expression}] "}" | ("not" | "¬" | "#" | "∁") factor
    term ::= factor {("×" | "div" | "mod" | "∩" | "and") factor}
    simpleExpression ::= ["+" | "-"] term {("+" | "-" | "∪" | "or") term}
    expression ::= simpleExpression
        {("=" | "≠" | "<" | "≤" | ">" | "≥" | "∈" | "⊆" | "⊇") simpleExpression}
    statementList ::= statement {";" statement}
    statementBlock ::= statementList {statementList}
    statementSuite ::= statementList | INDENT statementBlock DEDENT
    statement ::=
        ident selector ":=" expression |
        ident {"," ident} (":=" expression {"," expression} | ("←" | "<-") ident "(" [expression {"," expression}] ")") |
        "if" expression "then" statementSuite ["else" statementSuite] |
        "while" expression "do" statementSuite
    type ::=
        ident |
        "[" expression ".." expression "]" ("→" | "->") type |
        "(" typedIds ")" |
        "set" "[" expression ".." expression "]"
    typedIds ::= ident {"," ident} ":" type {"," ident {"," ident} ":" type}.
    declarations ::= 
        {"const" ident "=" expression}
        {"type" ident "=" type}
        {"var" typedIds}
        {"procedure" ident "(" [typedIds] ")" [ ("→" | "->") "(" typedIds ")" ] body}
    body ::= INDENT declarations (statementBlock | INDENT statementBlock DEDENT) DEDENT
    program ::= declarations "program" ident body

In [43]:
compileString("""
procedure quotrem(x, y: integer) -> (q, r: integer)
    q, r := 0, x
    while r ≥ y do // q × y + r = x ∧ r ≥ y
        r, q := r - y, q + 1

program arithmetic
    var x, y, q, r: integer
      x <- read(); y <- read()
      q, r <- quotrem(x, y)
      if not(q = r) then write(q)
""", "ascii.wat")

In [44]:
!wat2wasm ascii.wat

In [48]:
runwasm("ascii.wasm")

<IPython.core.display.Javascript object>

In [46]:
runpywasm("ascii.wasm")

 6
 3


AssertionError: 

In [47]:
runwasmer("ascii.wasm")

 6
 3


2 